In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings, OpenAIEmbeddings

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

memory = ConversationBufferMemory(return_messages=True,
                                  memory_key="chat_history",
                                  human_prefix="human",
                                  ai_prefix="ai")


def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]


cache_dir = LocalFileStore("./.cache/")

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir)

splitter = CharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100,
    separator="\n",
)

loader = UnstructuredFileLoader("./document.txt")

docs = loader.load_and_split(text_splitter=splitter)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retreiver = vectorstore.as_retriever()

Created a chunk of size 717, which is longer than the specified 600
Created a chunk of size 608, which is longer than the specified 600
Created a chunk of size 642, which is longer than the specified 600
Created a chunk of size 1444, which is longer than the specified 600
Created a chunk of size 1251, which is longer than the specified 600
Created a chunk of size 1012, which is longer than the specified 600
Created a chunk of size 1493, which is longer than the specified 600
Created a chunk of size 819, which is longer than the specified 600
Created a chunk of size 1458, which is longer than the specified 600
Created a chunk of size 1411, which is longer than the specified 600
Created a chunk of size 742, which is longer than the specified 600
Created a chunk of size 669, which is longer than the specified 600
Created a chunk of size 906, which is longer than the specified 600
Created a chunk of size 703, which is longer than the specified 600
Created a chunk of size 1137, which is lon

In [6]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system", """
     Given the following extracted parts of a long document and a question, create a final answer.
     If you don't know the answer, just say that you don't know. Don't try to make up an answer.
     ---------
     {context}
     """),
    ("human", "{question}"),
])

map_doc_prompt = ChatPromptTemplate.from_messages([("system", """
     Use the following portion of a long document to see if any of the text is relevant to answer the question. 
     Return any relevant text verbatim. If there is no relevant text return: ''
     -------
     {context}
     """), ("human", "{question}")])

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke({
            "context": document.page_content,
            "question": question
        }).content for document in documents)


map_chain = {
    "documents": retreiver,
    "question": RunnablePassthrough()
} | RunnableLambda(map_docs)

chain = {
    "context": map_chain,
    "question": RunnablePassthrough()
} | final_prompt | llm

In [7]:
print(chain.invoke("Is Aaronson guilty?"))

content='Yes, Aaronson was guilty of the crimes he was charged with.'


In [8]:
print(chain.invoke("What message did he write in the table?"))

content="I don't know."


In [9]:
print(chain.invoke("Who is Julia?"))

content='Julia is a character mentioned in the text provided.'
